El siguiente código tiene como propósito estudiar la probabilidad de sobrevivencia de los pasajeros del Titanic.

El objetivo es crear un modelo de clasificación binaria usando la técnica de gradiant boosting.

Se importan la librerias a utilizar.
En primer lugar librerías de tratamiento de datos. Luego librerías para gráficos, para luego importar las librerías propias del modelamiento.

In [ ]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

Se hace la llamada a la información para y se da forma al dataframe inicial.

In [ ]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
titanic_df = pd.read_csv(url)

In [ ]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Se divide el set de datos en dos: uno de entrenamiento (df_train) y otro para testear el modelo entrenado.

In [ ]:
df_train, df_test = train_test_split(titanic_df, test_size=0.2, random_state=42)

print(df_train.shape)
print(df_test.shape)

(712, 12)
(179, 12)


Feature engineering

Para ejecutar correctamente un gradiant boosting, se debe tener información en cada valor de cada variable, es por ello que se debe imputar según sea la naturaleza de la variable.

1. Se cambian los datos de sexos en números.
2. Se cambian los datos de embarque en números

In [ ]:

df_train['Sex'].replace(['female','male'],[0,1],inplace=True)
df_test['Sex'].replace(['female','male'],[0,1],inplace=True)


df_train['Embarked'].replace(['Q','S', 'C'],[0,1,2],inplace=True)
df_test['Embarked'].replace(['Q','S', 'C'],[0,1,2],inplace=True)

<ipython-input-5-377eb15c3488>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Sex'].replace(['female','male'],[0,1],inplace=True)
<ipython-input-5-377eb15c3488>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['Sex'].replace(['female','male'],[0,1],inplace=True)
<ipython-input-5

3. Se reemplaza los datos faltantes en la edad por la media de esta columna.
4. Se crea una variable tramificada para la edad, según las siguientes bandas: 0-8, 9-15, 16-18, 19-25, 26-40, 41-60, 61-100

In [ ]:

print(df_train["Age"].mean())
print(df_test["Age"].mean())
promedio = 30
df_train['Age'] = df_train['Age'].replace(np.nan, promedio)
df_test['Age'] = df_test['Age'].replace(np.nan, promedio)

#Tramos para edad
bins = [0, 8, 15, 18, 25, 40, 60, 100]
names = ['1', '2', '3', '4', '5', '6', '7']
df_train['Age'] = pd.cut(df_train['Age'], bins, labels = names)
df_test['Age'] = pd.cut(df_test['Age'], bins, labels = names)

29.498846153846156
30.505845070422534


5. Se elimina la columna de "Cabin" ya que tiene muchos datos perdidos.
6. Elimino las columnas que considero que no son necesarias para el analisis.
7. Se elimina las filas con los datos perdidos. Y se verifica el resultado.

In [ ]:

df_train.drop(['Cabin'], axis = 1, inplace=True)
df_test.drop(['Cabin'], axis = 1, inplace=True)

df_train = df_train.drop(['PassengerId','Name','Ticket'], axis=1)
df_test = df_test.drop(['Name','Ticket'], axis=1)

df_train.dropna(axis=0, how='any', inplace=True)
df_test.dropna(axis=0, how='any', inplace=True)

#Verifico los datos
print(pd.isnull(df_train).sum())
print(pd.isnull(df_test).sum())

print(df_train.shape)
print(df_test.shape)

print(df_test.head())
print(df_train.head())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
(710, 8)
(179, 9)
     PassengerId  Survived  Pclass  Sex Age  SibSp  Parch     Fare  Embarked
709          710         1       3    1   5      1      1  15.2458         2
439          440         0       2    1   5      0      0  10.5000         1
840          841         0       3    1   4      0      0   7.9250         1
720          721         1       2    0   1      0      1  33.0000         1
39            40         1       3    0   2      1      0  11.2417         2
     Survived  Pclass  Sex Age  SibSp  Parch     Fare  Embarked
331         0       1    1   6      0      0  28.5000       1.0
733         0       2    1   4      0      0  13.0000       1.0
382         0       3    1   5      0

Separo la columna con la información de los y las sobrevivientes tanto en el target como el dataset.

In [ ]:

#X = np.array(df_train.drop(['Survived'], 1))
X = np.array(df_train.drop(columns='Survived'))
y = np.array(df_train['Survived'])

#Separo los datos de "train" en entrenamiento y prueba para probar los algoritmos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Se entrena el modelo vía xgboost y se entrega el AUC para evaluar la calidad del entrenamiento.

In [ ]:
xgb = XGBClassifier()

xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
xgb.score(X_test, y_test)

0.8028169014084507

Conclusión:

Se logra entrenar un modelo de clasificación binaria utilizando un clasificador adecuado para dicho ejercicio.